<center><font size="6">Хакатон от Моторики</font>

[Третий этап](https://www.kaggle.com/competitions/motorica-advanced-gesture-classification)

Команда 1

**Задача:**  Построить классификационную модель, которая по показаниям оптомиографических датчиков будет определять жест оператора протеза кисти во время записи данных, что отражено в значении целевой переменной. 

Имеется три оператора, выполнявшие некоторую последовательность жестов. В результате наблюдений получены показания датчиков в виде непрерывного временного ряда. 

Список файлов, в котором символ * обозначает индекс оператора:
* X_train_*.npy - обучающая выборка, имеющая размерности "наблюдения, датчики, время";
* X_test_dataset_*.pkl - тестовая выборка, имеющая размерности "наблюдения, датчики, время";
* sample_submission.csv - некоторое решение задачи, из которого можно взять формат загрузки данных на Kaggle;
* y_train_*.npy - целевая переменная для обучающей выборки, которая содержит следующие классы:

| Номер <br> класса | Описание жеста |
| :---: | :--- |
| "-1"  |  потенциально "битые" данные |
| "0"  |  жест "open" - разгиб всех пальцев |
| "1"  |  жест "пистолет" |
| "2"  |  сгиб большого пальца |
| "3"  |  жест "ok" |
| "4"  |  жест "grab" |

* functions.py - библиотека функций, применяемых в данном ноутбуке.

# Import библиотек и описание функций

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
import copy
import pickle

# графические библиотеки
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# библиотеки машинного обучения
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# отображать по умолчанию длину дата Датафрейма
pd.set_option("display.max_rows", 11, "display.max_columns", 11)

# библиотека взаимодействия с интерпретатором
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import os

# отображение элементов массива не в научном формате
np.set_printoptions(suppress=True)

In [2]:
# библиотека вызова функций
import functions as f

# константы и глобальные параметры данных
PATH = 'data/'
GEST = 5          # количество основных жестов (кроме 'битых' данных)
PL = 3            # количество пилотов (независимых рядов данных)
SENS = 50         # количество датчиков

X_train = ['' for p in range(PL)]  # список объектов X_train
y_train = ['' for p in range(PL)]  # список объектов y_train
X_test = ['' for p in range(PL)]   # список объектов X_test

t_train = ['' for p in range(PL)]   # список длин X_train и y_train (кол-во временных тиков)
t_test = [0 for p in range(PL)]     # список с общим количеством временных тиков в X_test
s_test = ['' for p in range(PL)]    # список с количеством событий в X_test
tt_test = [[] for p in range(PL)]   # список списков с длинами событий в X_test

*Пример вызова функции из файла. Если добавленная Вами функция не работает, то перезапустите ядро*

# Загрузка данных
---

In [3]:
# Файлы можно хранить в архиве и читать прямо из него.

import zipfile
archive = zipfile.ZipFile('data/motorica-advanced-gesture-classification.zip', 'r')

X_test_dataset_1 = archive.read('X_test_dataset_1.pkl')
X_test_dataset_2 = archive.read('X_test_dataset_2.pkl')
X_test_dataset_3 = archive.read('X_test_dataset_3.pkl')
X_train_1 = archive.read('X_train_1.npy')
X_train_2 = archive.read('X_train_2.npy')
X_train_3 = archive.read('X_train_3.npy')
y_train_1 = archive.read('y_train_1.npy')
y_train_2 = archive.read('y_train_2.npy')
y_train_3 = archive.read('y_train_3.npy')

In [4]:
mounts = {
    1 : {
        'path_X_train' : 'X_train_1.npy',
        'path_y_train' : 'y_train_1.npy',
        'path_X_test_dataset' : 'X_test_dataset_1.pkl',
    },
    2 : {
        'path_X_train' : 'X_train_2.npy',
        'path_y_train' : 'y_train_2.npy',
        'path_X_test_dataset' : 'X_test_dataset_2.pkl',
    },
    3 : {
        'path_X_train' : 'X_train_3.npy',
        'path_y_train' : 'y_train_3.npy',
        'path_X_test_dataset' : 'X_test_dataset_3.pkl',
    }
}

In [5]:
path_to_zip = '.\data\motorica-advanced-gesture-classification.zip'
from zipfile import *
for mount_name, mount in mounts.items():
    mount['X_train'] = np.load(path_to_zip)[mount['path_X_train']]
    mount['y_train'] = np.load(path_to_zip)[mount['path_y_train']]
    with ZipFile('.\data\motorica-advanced-gesture-classification.zip') as myzip:
        with myzip.open(mount['path_X_test_dataset']) as myfile:
            mount['X_test_dataset'] = pickle.load(myfile)
    print(len(mount['X_test_dataset']))

899
855
861


In [6]:
# загрузка обучающей выборки и меток классов, тестовой выборки
# X_test_raw = ['' for p in range(PL)]
# for p in range(PL):
#     X_train[p] = np.load(os.path.join(PATH, 'X_train_' + f'{p+1}' + '.npy'))
#     t_train[p] = X_train[p].shape[0]
    
#     y_train[p] = np.load(os.path.join(PATH, 'y_train_' + f'{p+1}' + '.npy'))
#     if y_train[p].shape[0] == t_train[p]:
#         print(f'длина обучающей выборки пилота №{p+1} = {t_train[p]}')
#     else:
#         print(f'длина обучающей выборки и меток классов пилота №{p+1} не совпадают!')
    
#     with open(os.path.join(PATH, 'X_test_dataset_' + f'{p+1}' + '.pkl'), 'rb') as file:
#         X_test_raw[p] = pickle.load(file)

In [7]:
# загрузка обучающей выборки и меток классов, тестовой выборки
X_test_raw = ['' for p in range(PL)]
for p in range(PL):
    X_train[p] = mounts[p+1]['X_train']
    t_train[p] = mounts[p+1]['X_train'].shape[0]
    
    y_train[p] = mounts[p+1]['y_train']
    
    if y_train[p].size == t_train[p]:
        print(f'длина обучающей выборки пилота №{p+1} = {t_train[p]}')
    else:
        print(f'длина обучающей выборки и меток классов пилота №{p+1} не совпадают!')
    
    #with open(os.path.join(PATH, 'X_test_dataset_' + f'{p+1}' + '.pkl'), 'rb') as file:
    X_test_raw[p] = mount['X_test_dataset'][p]

длина обучающей выборки пилота №1 = 24030
длина обучающей выборки пилота №2 = 23202
длина обучающей выборки пилота №3 = 23177


In [8]:
mounts[1]['X_train']

array([[2711,   16, 1803, ...,    8,   14,    7],
       [2712,   10, 1802, ...,    6,   12,    6],
       [2711,   13, 1803, ...,    5,   16,    8],
       ...,
       [2707,   16, 1851, ...,    6,   13,    6],
       [2707,   18, 1850, ...,    8,   12,    6],
       [2710,   16, 1851, ...,    9,   11,    8]], dtype=int64)

In [9]:
# смотрим на исходные данные
print('Тип данных X_train: ', type(X_train[0]))
print(f'Размерность файла X_train_1: {X_train[0].shape} - время, датчики')
print('X_train первого пилота:')
X_train[0]

Тип данных X_train:  <class 'numpy.ndarray'>
Размерность файла X_train_1: (24030, 50) - время, датчики
X_train первого пилота:


array([[2711,   16, 1803, ...,    8,   14,    7],
       [2712,   10, 1802, ...,    6,   12,    6],
       [2711,   13, 1803, ...,    5,   16,    8],
       ...,
       [2707,   16, 1851, ...,    6,   13,    6],
       [2707,   18, 1850, ...,    8,   12,    6],
       [2710,   16, 1851, ...,    9,   11,    8]], dtype=int64)

In [10]:
print('Тип данных y_train: ', type(y_train[0]))
print(f'Размерность файла y_train_1: {y_train[0].shape} - время')
print('y_train первого пилота:')
y_train[0]

Тип данных y_train:  <class 'numpy.ndarray'>
Размерность файла y_train_1: (24030,) - время
y_train первого пилота:


array([0, 0, 0, ..., 3, 3, 3], dtype=int64)

In [11]:
# Сосчитаем количество ненулевых жестов
test_id = 1 # номер наблюдения
counter=0
for i in range(len(y_train[test_id])):
    if y_train[test_id][i] !=y_train[test_id][i-1] and y_train[test_id][i] >=0:
        counter+=1
counter

260

In [12]:
# смотрим на исходные данные X_test
print(f"Тип данных X_test: {type(mount['X_test_dataset'])}")
print(f"Тип данных X_test для пилота 1: {type(mount['X_test_dataset'][1])}")
print(f"Размерность файла X_test_1: {mount['X_test_dataset'][1].shape} - время, датчики")
print(f"X_test первого пилота, наблюдение '0': \n {mount['X_test_dataset'][1][0]}")

Тип данных X_test: <class 'list'>
Тип данных X_test для пилота 1: <class 'numpy.ndarray'>
Размерность файла X_test_1: (50, 256) - время, датчики
X_test первого пилота, наблюдение '0': 
 [1370 1372 1369 1363 1361 1356 1354 1354 1353 1350 1352 1356 1359 1359
 1359 1358 1358 1360 1359 1359 1358 1356 1355 1355 1354 1355 1356 1359
 1363 1361 1362 1366 1365 1367 1366 1371 1386 1395 1395 1368 1342 1317
 1291 1272 1254 1235 1213 1194 1176 1163 1145 1131 1121 1111 1108 1106
 1108 1115 1119 1118 1117 1115 1112 1113 1113 1113 1111 1110 1110 1109
 1110 1108 1106 1111 1110 1111 1110 1113 1110 1109 1110 1111 1113 1112
 1112 1112 1112 1114 1114 1114 1116 1115 1118 1117 1113 1115 1116 1146
 1197 1236 1271 1291 1300 1303 1309 1317 1327 1332 1340 1349 1355 1361
 1367 1371 1371 1379 1387 1400 1408 1413 1415 1416 1421 1427 1429 1428
 1426 1425 1425 1420 1418 1415 1411 1408 1408 1406 1405 1407 1404 1405
 1404 1405 1401 1402 1402 1403 1402 1402 1403 1402 1401 1398 1398 1395
 1396 1397 1396 1395 1392 1391 13

In [13]:
print(f"Тип данных X_test: {type(mount['X_test_dataset'])}")
print(f"Тип данных X_test для пилота 1: {type(mount['X_test_dataset'][1])}")
print(f"Размерность файла X_test_1: {mount['X_test_dataset'][1].shape} - время, датчики")
print(f"X_test первого пилота, наблюдение '0': \n {mount['X_test_dataset'][1][0]}")

Тип данных X_test: <class 'list'>
Тип данных X_test для пилота 1: <class 'numpy.ndarray'>
Размерность файла X_test_1: (50, 256) - время, датчики
X_test первого пилота, наблюдение '0': 
 [1370 1372 1369 1363 1361 1356 1354 1354 1353 1350 1352 1356 1359 1359
 1359 1358 1358 1360 1359 1359 1358 1356 1355 1355 1354 1355 1356 1359
 1363 1361 1362 1366 1365 1367 1366 1371 1386 1395 1395 1368 1342 1317
 1291 1272 1254 1235 1213 1194 1176 1163 1145 1131 1121 1111 1108 1106
 1108 1115 1119 1118 1117 1115 1112 1113 1113 1113 1111 1110 1110 1109
 1110 1108 1106 1111 1110 1111 1110 1113 1110 1109 1110 1111 1113 1112
 1112 1112 1112 1114 1114 1114 1116 1115 1118 1117 1113 1115 1116 1146
 1197 1236 1271 1291 1300 1303 1309 1317 1327 1332 1340 1349 1355 1361
 1367 1371 1371 1379 1387 1400 1408 1413 1415 1416 1421 1427 1429 1428
 1426 1425 1425 1420 1418 1415 1411 1408 1408 1406 1405 1407 1404 1405
 1404 1405 1401 1402 1402 1403 1402 1402 1403 1402 1401 1398 1398 1395
 1396 1397 1396 1395 1392 1391 13

In [14]:
# транспонируем np.array для каждого события в X_test
t_test = [0 for p in range(PL)]
X_test = copy.deepcopy(X_test_raw)
for p in range(PL):
    s_test[p] = len(X_test_raw[p])
    print(f'Количество событий для пилота №{p+1} = {s_test[p]}')
    for s in range(s_test[p]):
        X_test[p][s] = X_test[p][s].T
        tt_test[p].append(len(X_test[p][s]))
        t_test[p] += tt_test[p][s]
print(f'X_test первого пилота содержит всего {t_test[0]} временных тиков')
print(f'X_test первого пилота, событие "0" длиной {tt_test[0][0]} тиков:')
X_test[0][0]

Количество событий для пилота №1 = 50
Количество событий для пилота №2 = 50
Количество событий для пилота №3 = 50
X_test первого пилота содержит всего 5250 временных тиков
X_test первого пилота, событие "0" длиной 105 тиков:


array([1130, 1130, 1132, 1129, 1129, 1128, 1128, 1126, 1128, 1124, 1124,
       1122, 1125, 1121, 1121, 1120, 1118, 1119, 1118, 1119, 1119, 1117,
       1118, 1116, 1116, 1116, 1116, 1116, 1117, 1116, 1113, 1111, 1108,
       1108, 1106, 1110, 1136, 1167, 1182, 1188, 1199, 1202, 1203, 1200,
       1174, 1158, 1155, 1149, 1134, 1126, 1121, 1116, 1115, 1111, 1108,
       1107, 1105, 1100, 1101, 1102, 1100, 1100, 1100, 1102, 1101, 1103,
       1103, 1102, 1099, 1099, 1101, 1102, 1108, 1118, 1135, 1152, 1157,
       1167, 1168, 1168, 1164, 1154, 1149, 1157, 1160, 1161, 1154, 1147,
       1146, 1140, 1138, 1136, 1136, 1133, 1132, 1128, 1128, 1120, 1119,
       1117, 1114, 1112, 1110, 1111, 1110], dtype=int64)

In [15]:
print('X_train первого пилота:')
X_train[0].shape

X_train первого пилота:


(24030, 50)

In [18]:
active_sensors, passive_sensor = f.get_sensor_list(1, mounts=mounts, print_active=True)

Активные датчики пилота 1:  [0, 2, 5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38]
Пассивные датчики пилота 1:  [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [24]:
f.get_all_sensors_plot(1, [100,300], mounts=mounts, plot_counter=1)

In [25]:
Pilot_id = 1
time_start=100
time_end=300

data = y_train
plot_counter=2

timesteps=[time_start, time_end]

df = pd.DataFrame(data = data[Pilot_id], index = [s for s in range(data[Pilot_id].shape[0])] 
                    #columns = [s for s in range(data[Pilot_id].shape[1])]
)

fig = px.line(data_frame=df.iloc[timesteps[0]:timesteps[1],:])

fig.update_layout(
    title=dict(text=f'Рис. {plot_counter}'+' - наблюдение ' + str(Pilot_id), x=.5, y=0.05, xanchor='center'), 
    xaxis_title_text = 'Время, сек', yaxis_title_text = 'Показатель',  yaxis_range = [0, 10],
    legend_title_text='Индекс <br>датчика',
    width=600, height=400,
    margin=dict(l=100, r=60, t=80, b=100),
)

fig.show()

In [26]:
active_sensors, passive_sensors = f.get_sensor_list(1, mounts=mounts, print_active=True)

Активные датчики пилота 1:  [0, 2, 5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38]
Пассивные датчики пилота 1:  [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [28]:
f.get_active_passive_sensors_plot(1, [100,300], mounts=mounts, plot_counter=3)

# Поиск периода времени, в котором изменился класс жеста

Проанализируем показания датчика 2

In [30]:
Pilot_id = 1
active_sensors, passive_sensors = f.get_sensor_list(1, mounts=mounts, print_active=True)
time_start = 250
time_end = 350
sensor = 2
plot_counter = 4

timesteps=[time_start, time_end]

df = pd.DataFrame(
    data = X_train[Pilot_id], index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]).iloc[timesteps[0]:timesteps[1],:]

df_2 = pd.DataFrame(data = y_train[Pilot_id], index = [s for s in range(y_train[Pilot_id].shape[0])]).iloc[timesteps[0]:timesteps[1],:]
    
df_1 = pd.DataFrame(df[sensor], columns=active_sensors)
#df_2 = pd.DataFrame(df[passive_sensors], columns=passive_sensors)


fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('X_train - активные датчики', 'y_train'), vertical_spacing = 0.1,
)

for i in df_1.columns: fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df[i].name)), row=1, col=1)

for i in df_2.columns: fig.add_trace(go.Scatter(x=df_2.index, y=df_2[i], name=str(df[i].name)), row=2, col=1)

fig.update_layout(title={'text':f'Рис. {plot_counter}'+' - Активные и пассивные датчики пилота ' + str(Pilot_id), 
'x':0.5, 'y':0.01}
)

fig.update_layout(width=600, height=800, legend_title_text ='Номер датчика',
                    xaxis_title_text  = 'Время',  yaxis_title_text = 'Сигнал датчика', yaxis_range=[1500, 1700], 
                    xaxis2_title_text = 'Время', yaxis2_title_text = 'Жест', #yaxis2_range= [0 , 200],
                    margin=dict(l=40, r=60, t=30, b=80), 
                    showlegend=False # легенда загромождает картинку
)

fig.show()

Активные датчики пилота 1:  [0, 2, 5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38]
Пассивные датчики пилота 1:  [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [42]:
Pilot_id = 2
#active_sensors, passive_sensors = f.get_sensor_list(1, X_train, print_active=True)
time_start = 100
time_end = 300
sensors = list([0, 2, #5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38
])

plot_counter = 5

timesteps=[time_start, time_end]

fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=(f'Исходный сигнал датчика ', f'Производная сигнала датчика ',
                    'y_train', f'Квадрат производной сигнала датчика '), vertical_spacing = 0.1,
)

df_1 = pd.DataFrame(
    data = X_train[Pilot_id], 
    index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]
).iloc[timesteps[0]:timesteps[1],:][sensors]
    
df_2 = pd.DataFrame(
    data = y_train[Pilot_id], 
    index = [s for s in range(y_train[Pilot_id].shape[0])]
).iloc[timesteps[0]:timesteps[1],:]

for i in df_1.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df_1[i].name)), row=1, col=1)



for i in df_2.columns: 
    fig.add_trace(go.Scatter(x=df_2.index, y=df_2[i], name=str(df_1[i].name)), row=2, col=1)

df_3 = pd.DataFrame(df_1.diff(), index = range(df_1.index[0], df_1.index[-1]+1))

for i in df_3.columns: 
    fig.add_trace(go.Scatter(x=df_3.index, y=df_3[i], name=str(df_1[i].name)), row=1, col=2)

#  датасет квадрата производной
df_4 = pd.DataFrame(np.power(df_1.diff(),2), index = range(df_1.index[0], df_1.index[-1]+1))

for i in df_4.columns: 
    fig.add_trace(go.Scatter(x=df_4.index, y=df_4[i], name=str(df_1[i].name)), row=2, col=2)

fig.update_layout(title={'text':f'Рис. {plot_counter} - Преобразование сигнала датчиков {sensors} пилота {Pilot_id}', 'x':0.5, 'y':0.01}
)

fig.update_layout(width=1200, height=800, legend_title_text =f'Номер датчика ',
                    xaxis_title_text  = 'Время',  yaxis_title_text = 'Сигнал датчика', #yaxis_range=[0, 4000], 
                    xaxis2_title_text = 'Время', yaxis2_title_text = 'Жест', #yaxis2_range= [0 , 8],
                    xaxis3_title_text = 'Время', yaxis3_title_text = 'Жест', yaxis3_range= [-1 , 8],
                    xaxis4_title_text = 'Время', yaxis4_title_text = 'Сигнал', #yaxis2_range= [0 , 8],
                    margin=dict(l=40, r=60, t=30, b=80), 
                    showlegend=False # легенда загромождает картинку
)

fig.show()

In [39]:
df_1

,0,2
100,1084,1238
101,1085,1239
102,1086,1240
103,1088,1241
...,...,...
296,1260,1377
297,1259,1376
298,1259,1376
299,1259,1376


In [34]:
X_train[0]

array([[2711,   16, 1803, ...,    8,   14,    7],
       [2712,   10, 1802, ...,    6,   12,    6],
       [2711,   13, 1803, ...,    5,   16,    8],
       ...,
       [2707,   16, 1851, ...,    6,   13,    6],
       [2707,   18, 1850, ...,    8,   12,    6],
       [2710,   16, 1851, ...,    9,   11,    8]], dtype=int64)

In [ ]:
mounts[1]['X_train']

# Поиск периода времени, в котором изменился класс жеста

Проанализируем показания датчика 2

In [ ]:
X_train[0].shape[1]

In [ ]:
Pilot_id = 1
time_start = 100
time_end = 600
sensors = list([0, 2, #5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38
])

plot_counter = 6

timesteps=[time_start, time_end]


df_1 = pd.DataFrame(
    data = X_train[Pilot_id], 
    index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]
    ).iloc[timesteps[0]:timesteps[1],:][sensors]
    
# Нормализация данных
scaler = StandardScaler()
scaler.fit(df_1)
df_1 = pd.DataFrame(scaler.transform(df_1))

fig = make_subplots(rows=2, cols=2, 
    subplot_titles=(f'X_train - нормализованные сигналы',  f'Производная нормализованных сигналов датчиков',
    'y_train', f'Квадрат производной нормализованных сигналов датчиков'), vertical_spacing = 0.1,
)

for i in df_1.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df_1[i].name)), row=1, col=1)

df_2 = pd.DataFrame(data = y_train[Pilot_id], index = [s for s in range(y_train[Pilot_id].shape[0])]).iloc[timesteps[0]:timesteps[1],:]

for i in df_2.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_2[i], name=str(df[i].name)), row=2, col=1)


df_3 = pd.DataFrame(df_1.diff(), index = range(df_1.index[0], df_1.index[-1]+1))


for i in df_3.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df_1[i].name)), row=1, col=2)

#  датасет квадрата производной
df_4 = pd.DataFrame(np.power(df_1.diff(),2), index = range(df_1.index[0], df_1.index[-1]+1))

for i in df_4.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_4[i], name=str(df_4[i].name)), row=2, col=2)

fig.update_layout(title={'text':f'Рис. {plot_counter} - Нормализованные сигналы датчиков {sensors} пилота {Pilot_id}', 
'x':0.5, 'y':0.01}
)

fig.update_layout(width=1200, height=800, legend_title_text ='Номер датчика',
                    xaxis_title_text  = 'Время',  yaxis_title_text = 'Сигнал датчика', #yaxis_range=[1500, 1700], 
                    xaxis2_title_text = 'Время', yaxis2_title_text = 'Жест', #yaxis2_range= [0 , 200],
                    margin=dict(l=40, r=60, t=30, b=80), 
                    showlegend=False # легенда загромождает картинку
)

fig.show()

In [ ]:
Pilot_id=1
timesteps=[100,450]
sensors=[0,2]


df_1 = pd.DataFrame(
    data = X_train[Pilot_id], 
    index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]
).iloc[timesteps[0]:timesteps[1],:][sensors]

df_1

In [ ]:
Pilot_id = 1
time_start = 100
time_end = 450
sensors = list([0, 2, #5, 8, 12, 15, 17, 19, 21, 24, 27, 29, 30, 33, 36, 38
])

plot_counter = 7

timesteps=[time_start, time_end]

fig = make_subplots(rows=4, cols=2, 
                    subplot_titles=(
                    f'Исходные сигналы датчиков', f'Нормализованные сигналы датчиков',
                    'y_train', 'y_train',
                    f'Производная сигналов датчиков', f'Производная нормализованных сигналов датчиков', 
                    f'Квадрат производной сигналов датчиков', f'Квадрат производной нормализованных сигналов датчиков'
                    ), vertical_spacing = 0.1,
)

df_1 = pd.DataFrame(
    data = X_train[Pilot_id], 
    index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]
    ).iloc[timesteps[0]:timesteps[1],:][sensors]

for i in df_1.columns: 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df[i].name)), row=1, col=1)

df_2 = pd.DataFrame(data = y_train[Pilot_id], index = [s for s in range(y_train[Pilot_id].shape[0])]).iloc[timesteps[0]:timesteps[1],:]

for i in df_2.columns: 
    fig.add_trace(go.Scatter(x=df_2.index, y=df_2[i], name=str(df[i].name)), row=2, col=1)

df_3 = pd.DataFrame(df_1.diff(), index = range(df_1.index[0], df_1.index[-1]+1))

for i in df_3.columns: 
    fig.add_trace(go.Scatter(x=df_3.index, y=df_3[i], name=str(df[i].name)), row=3, col=1)

#  датасет квадрата производной
df_4 = pd.DataFrame(np.power(df_1.diff(),2), index = range(df_1.index[0], df_1.index[-1]+1))

for i in df_4.columns: 
    fig.add_trace(go.Scatter(x=df_4.index, y=df_4[i], name=str(df[i].name)), row=4, col=1)

# Нормализация данных
scaler = StandardScaler()
scaler.fit(df_1)
df_5 = pd.DataFrame(scaler.transform(df_1))

for i in df_5.columns: 
    fig.add_trace(go.Scatter(x=df_5.index, y=df_5[i], name=str(df_5[i].name)), row=1, col=2)

#df_6 = pd.DataFrame(data = y_train[Pilot_id], index = [s for s in range(y_train[Pilot_id].shape[0])]).iloc[timesteps[0]:timesteps[1],:]

for i in df_2.columns: 
    fig.add_trace(go.Scatter(x=df_2.index, y=df_2[i], name=str(df[i].name)), row=2, col=2)


df_7 = pd.DataFrame(df_5.diff(), index = range(df_5.index[0], df_5.index[-1]+1))


for i in df_7.columns: 
    fig.add_trace(go.Scatter(x=df_5.index, y=df_7[i], name=str(df_5[i].name)), row=3, col=2)

#  датасет квадрата производной
df_8 = pd.DataFrame(np.power(df_5.diff(),2), index = range(df_5.index[0], df_5.index[-1]+1))

for i in df_8.columns: 
    fig.add_trace(go.Scatter(x=df_5.index, y=df_8[i], name=str(df_5[i].name)), row=4, col=2)

fig.update_layout(title={'text':f'Рис. {plot_counter} - Преобразование сигнала датчиков {sensors} пилота {Pilot_id}', 'x':0.5, 'y':0.01}
)

fig.update_layout(width=1200, height=1200, legend_title_text =f'Номер датчика ',
                    xaxis_title_text  = 'Время',  yaxis_title_text = 'Сигнал датчика', #yaxis_range=[0, 4000], 
                    xaxis2_title_text = 'Время', yaxis2_title_text = 'Жест', #yaxis2_range= [0 , 8],
                    xaxis3_title_text = 'Время', yaxis3_title_text = 'Жест', #yaxis3_range= [-1 , 8],
                    xaxis4_title_text = 'Время', yaxis4_title_text = 'Жест', #yaxis2_range= [0 , 8],
                    margin=dict(l=40, r=60, t=30, b=80), 
                    showlegend=False # легенда загромождает картинку
)

fig.show()

Апроксимируем сигналы датчиков некоторой функцией, которая принимает среднее значение сигнала в течении заданного периода.

In [ ]:
# Номер пилота
Pilot_id = 1
# начальный и конечный периоды времени
time_start = 100
time_end = 400
sensors = list([0, 
#2, 5, 8, 12, 15, 
#17, 19, 21, 24, 27, 29, 30, 33, 36, 38
])
# шаг апрокимации
aproximation_step = 70

plot_counter = 8

timesteps=[time_start, time_end]


df_1 = pd.DataFrame(
    data = X_train[Pilot_id], 
    index = [s for s in range(X_train[Pilot_id].shape[0])], 
    columns = [s for s in range(X_train[Pilot_id].shape[1])]
    ).iloc[timesteps[0]:timesteps[1],:][sensors]
#display(df_1)

# создаем пустой массив такой же размерности, как оригинальный
df_empty = pd.DataFrame(index = range(df_1.index[0], df_1.index[-1]+1), columns = df_1.columns)



for i in range(0,df_1.shape[0],aproximation_step):
    df_empty.iloc[i:i+aproximation_step,:] = df_1.iloc[i:i+aproximation_step,:].mean()

fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('X_train - активные датчики', 'y_train'), vertical_spacing = 0.1,
)

for i in df_empty.columns:
    # апроксимированный сигнал 
    fig.add_trace(go.Scatter(x=df_1.index, y=df_empty[i], name=str(df[i].name)), row=1, col=1)
    # исходный сигнал
    #fig.add_trace(go.Scatter(x=df_1.index, y=df_1[i], name=str(df[i].name)), row=1, col=1)

df_2 = pd.DataFrame(data = y_train[Pilot_id], index = [s for s in range(y_train[Pilot_id].shape[0])]).iloc[timesteps[0]:timesteps[1],:]

for i in df_2.columns: fig.add_trace(go.Scatter(x=df_2.index, y=df_2[i], name=str(df[i].name)), row=2, col=1)

fig.update_layout(title={'text':f'Рис. {plot_counter} - Активные и пассивные датчики пилота {Pilot_id}', 
'x':0.5, 'y':0.01}
)

fig.update_layout(width=600, height=800, legend_title_text ='Номер датчика',
                    xaxis_title_text  = 'Время',  yaxis_title_text = 'Сигнал датчика', #yaxis_range=[1500, 1700], 
                    xaxis2_title_text = 'Время', yaxis2_title_text = 'Жест', yaxis2_range= [-1 , 8],
                    margin=dict(l=40, r=60, t=30, b=80), 
                    showlegend=True # легенда загромождает картинку
)

fig.show()

In [ ]:
# Анализируем датчик 8 для рассматриваемого наблюдения 2
plot_counter=2

id = 2
time_stp = 0 # начальный интервал времени можно сдвинуть
sensor = 24
df = pd.DataFrame(data = X_train[id], index = [s for s in range(X_train.shape[1])], 
                  columns = [s for s in range(X_train.shape[2])]
).T

y_k = y_train[id*100:(id+1)*100].reset_index().T
# временной промежуток
time_stp = 0
time_end = 100

df_T_50 = df.iloc[time_stp:]
y_k_50 = y_k[range(time_stp, 100)]
#df_T_50 = df_T_50.rolling(window=5).mean().dropna(axis=0)

f, ax = plt.subplots(2, 2, figsize = (20, 9))
plt.subplots_adjust(  left=0.1,   right=0.9,
                    bottom=0.1,     top=0.9,
                    wspace=0.1,  hspace=0.4)

plt.suptitle(f'Рис. {plot_counter}'+' - сравнение и преобразование сигнала датчика', y=-0.01, fontsize=16)

ax[0,0].plot(list(y_k_50.columns), y_k_50.loc['class'].values)
ax[0,0].set_title(f'Изменение класса жеста', fontsize=16)
ax[0,0].set_ylabel('Класс жеста')

ax[1,0].plot(list(df_T_50.index), df_T_50[sensor].values)
ax[1,0].set_title(f'Сигнал датчика {sensor} в наблюдении {id}', fontsize=16)
ax[1,0].set_ylabel('Сигнал датчика')

ax[0,1].plot(list(range(len(np.diff(df_T_50[sensor], 1)))), np.diff(df_T_50[sensor],1))
ax[0,1].set_title(f'Производная сигнала датчика {sensor}', fontsize=16)
ax[0,1].grid()

ax[1,1].plot(list(range(len(np.diff(df_T_50[sensor], 1)))), np.power(np.diff(df_T_50[sensor],1),2))
ax[1,1].set_title(f'Квадрат производной сигнала датчика {sensor}', fontsize=16)
ax[1,1].grid()


plt.show()

In [ ]:
arr = np.array(df.iloc[:,2], 'int64')
arr

In [ ]:
arr = np.array(df.iloc[:,2], 'int16')
a = np.array([], 'int16')

for i in range(arr.size):
    if i==0:
        a = np.append(a,0)
    

    else:
        b = (df.iloc[i,2] - df.iloc[i-1,2])
        a = np.append(a,b)
print(a)
print(a.size)

In [ ]:
df = pd.DataFrame(a)
#display(df)

fig = px.line(
    data_frame=df, #DataFrame
    #x='date', #ось абсцисс
    #y=['confirmed', 'recovered', 'deaths', 'active'], #ось ординат
    height=500, #высота
    width=1000, #ширина
    title='chart' #заголовок
)
fig.show()

In [ ]:
def plotMovingAverage(series, n):

    """
    series - dataframe with timeseries
    n - rolling window size 

    """

    rolling_mean = series.rolling(window=n).mean()

    # При желании, можно строить и доверительные интервалы для сглаженных значений
    #rolling_std =  series.rolling(window=n).std()
    #upper_bond = rolling_mean+1.96*rolling_std
    #lower_bond = rolling_mean-1.96*rolling_std

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(n))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    #plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
    #plt.plot(lower_bond, "r--")
    plt.plot(series[n:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [ ]:
plotMovingAverage(df[2], 5)